In [1]:
%run setting.ipynb

In [2]:
import lightgbm as lgb
import xgboost as xgb
import catboost as ctb

In [3]:
from sklearn import preprocessing
from sklearn.metrics import *

# training and test split date

In [4]:
test_start_date = '2022-01-01'

# load features

In [5]:
feature_name = np.load(
    os.path.join(
    data_path,
    'feature_name.npy',
    ),
)

categorical_feature = np.load(
    os.path.join(
    data_path,
    'categorical_feature.npy',
    ),
)

numemrical_feature = np.load(
    os.path.join(
    data_path,
    'numemrical_feature.npy',
    ),
)

# load data

In [6]:
data_filled = pd.read_parquet(
    os.path.join(
    data_path,
    'data_filled.parquet',
    )
)

data_encoded = pd.read_parquet(
    os.path.join(
    data_path,
    'data_encoded.parquet',
    )
)

In [7]:
X = data_filled[feature_name]

In [8]:
y = data_filled['label__rise']

In [9]:
Date = data_filled['Date']

In [10]:
data_encoded.drop(columns = ['level_0']).to_sql(
    "data_encoded", 
    conn,
    if_exists = "replace",
    )

9514

In [11]:
data_filled.drop(columns = ['level_0']).to_sql(
    "data_filled", 
    conn,
    if_exists = "replace",
    )

9514

# catboost model

## training test split

In [12]:
data_test = sql(f"""
select * 
from data_filled
where date(Date) >= '{test_start_date}'
""")

data_training = sql(f"""
select * 
from data_filled
where date(Date) < '{test_start_date}'
""")

X_training = data_training[feature_name]
y_training = data_training['label__rise']


X_test = data_test[feature_name]
y_test = data_test['label__rise']

In [13]:
print(f"""
trianing size:\t{X_training.shape}
test size:\t{X_test.shape}
""")


trianing size:	(9139, 72)
test size:	(375, 72)



## model training

In [14]:
%%time  

train_pool = ctb.Pool(
    X_training, 
    y_training, 
    cat_features = categorical_feature,
    )

params = {
    'loss_function': 'Logloss',
    'task_type':"GPU",
    'devices':'0',
    }

ctb_model = ctb.CatBoostClassifier(**params)

ctb_model.fit(train_pool)

ctb_model.save_model(
    fname = os.path.join(
    data_path,
    "daily_rise.catboost"),
    )

Learning rate set to 0.029849
0:	learn: 0.6926783	total: 48ms	remaining: 48s
1:	learn: 0.6922956	total: 97.6ms	remaining: 48.7s
2:	learn: 0.6916287	total: 149ms	remaining: 49.6s
3:	learn: 0.6910752	total: 201ms	remaining: 50s
4:	learn: 0.6907285	total: 269ms	remaining: 53.6s
5:	learn: 0.6901799	total: 322ms	remaining: 53.3s
6:	learn: 0.6898395	total: 380ms	remaining: 53.9s
7:	learn: 0.6893764	total: 442ms	remaining: 54.9s
8:	learn: 0.6890306	total: 489ms	remaining: 53.8s
9:	learn: 0.6885090	total: 540ms	remaining: 53.5s
10:	learn: 0.6881595	total: 596ms	remaining: 53.6s
11:	learn: 0.6877952	total: 647ms	remaining: 53.3s
12:	learn: 0.6873090	total: 694ms	remaining: 52.7s
13:	learn: 0.6868729	total: 742ms	remaining: 52.3s
14:	learn: 0.6865390	total: 790ms	remaining: 51.9s
15:	learn: 0.6861857	total: 836ms	remaining: 51.4s
16:	learn: 0.6859504	total: 882ms	remaining: 51s
17:	learn: 0.6856230	total: 932ms	remaining: 50.8s
18:	learn: 0.6853612	total: 978ms	remaining: 50.5s
19:	learn: 0.6850

162:	learn: 0.6567189	total: 7.83s	remaining: 40.2s
163:	learn: 0.6565308	total: 7.88s	remaining: 40.2s
164:	learn: 0.6564529	total: 7.92s	remaining: 40.1s
165:	learn: 0.6563589	total: 7.97s	remaining: 40.1s
166:	learn: 0.6562707	total: 8.02s	remaining: 40s
167:	learn: 0.6561801	total: 8.06s	remaining: 39.9s
168:	learn: 0.6560336	total: 8.11s	remaining: 39.9s
169:	learn: 0.6558260	total: 8.16s	remaining: 39.8s
170:	learn: 0.6557329	total: 8.21s	remaining: 39.8s
171:	learn: 0.6556046	total: 8.25s	remaining: 39.7s
172:	learn: 0.6554623	total: 8.3s	remaining: 39.7s
173:	learn: 0.6551651	total: 8.35s	remaining: 39.6s
174:	learn: 0.6549924	total: 8.39s	remaining: 39.6s
175:	learn: 0.6548689	total: 8.44s	remaining: 39.5s
176:	learn: 0.6546347	total: 8.48s	remaining: 39.4s
177:	learn: 0.6545181	total: 8.53s	remaining: 39.4s
178:	learn: 0.6543499	total: 8.58s	remaining: 39.3s
179:	learn: 0.6542400	total: 8.62s	remaining: 39.3s
180:	learn: 0.6540571	total: 8.67s	remaining: 39.2s
181:	learn: 0.6

322:	learn: 0.6364847	total: 15.2s	remaining: 31.9s
323:	learn: 0.6364746	total: 15.3s	remaining: 31.9s
324:	learn: 0.6364381	total: 15.3s	remaining: 31.8s
325:	learn: 0.6364372	total: 15.4s	remaining: 31.8s
326:	learn: 0.6362326	total: 15.4s	remaining: 31.7s
327:	learn: 0.6361572	total: 15.5s	remaining: 31.7s
328:	learn: 0.6361348	total: 15.5s	remaining: 31.6s
329:	learn: 0.6360874	total: 15.5s	remaining: 31.6s
330:	learn: 0.6359757	total: 15.6s	remaining: 31.5s
331:	learn: 0.6359410	total: 15.6s	remaining: 31.5s
332:	learn: 0.6359285	total: 15.7s	remaining: 31.4s
333:	learn: 0.6358505	total: 15.7s	remaining: 31.4s
334:	learn: 0.6357745	total: 15.8s	remaining: 31.3s
335:	learn: 0.6356144	total: 15.8s	remaining: 31.3s
336:	learn: 0.6354820	total: 15.9s	remaining: 31.2s
337:	learn: 0.6352505	total: 15.9s	remaining: 31.2s
338:	learn: 0.6351489	total: 16s	remaining: 31.1s
339:	learn: 0.6349458	total: 16s	remaining: 31.1s
340:	learn: 0.6349129	total: 16s	remaining: 31s
341:	learn: 0.634879

485:	learn: 0.6176959	total: 22.6s	remaining: 23.9s
486:	learn: 0.6175240	total: 22.7s	remaining: 23.9s
487:	learn: 0.6174589	total: 22.7s	remaining: 23.8s
488:	learn: 0.6172576	total: 22.7s	remaining: 23.8s
489:	learn: 0.6172571	total: 22.8s	remaining: 23.7s
490:	learn: 0.6171357	total: 22.8s	remaining: 23.7s
491:	learn: 0.6170162	total: 22.9s	remaining: 23.6s
492:	learn: 0.6169627	total: 22.9s	remaining: 23.6s
493:	learn: 0.6169017	total: 23s	remaining: 23.5s
494:	learn: 0.6169004	total: 23s	remaining: 23.5s
495:	learn: 0.6167548	total: 23s	remaining: 23.4s
496:	learn: 0.6167547	total: 23.1s	remaining: 23.4s
497:	learn: 0.6166466	total: 23.1s	remaining: 23.3s
498:	learn: 0.6165933	total: 23.2s	remaining: 23.3s
499:	learn: 0.6164272	total: 23.2s	remaining: 23.2s
500:	learn: 0.6164272	total: 23.2s	remaining: 23.1s
501:	learn: 0.6162790	total: 23.3s	remaining: 23.1s
502:	learn: 0.6162786	total: 23.3s	remaining: 23s
503:	learn: 0.6160770	total: 23.4s	remaining: 23s
504:	learn: 0.6160028	

646:	learn: 0.6029355	total: 29.5s	remaining: 16.1s
647:	learn: 0.6029348	total: 29.6s	remaining: 16.1s
648:	learn: 0.6028053	total: 29.6s	remaining: 16s
649:	learn: 0.6027646	total: 29.7s	remaining: 16s
650:	learn: 0.6027181	total: 29.7s	remaining: 15.9s
651:	learn: 0.6026403	total: 29.8s	remaining: 15.9s
652:	learn: 0.6024936	total: 29.8s	remaining: 15.8s
653:	learn: 0.6023349	total: 29.9s	remaining: 15.8s
654:	learn: 0.6021748	total: 29.9s	remaining: 15.7s
655:	learn: 0.6021722	total: 29.9s	remaining: 15.7s
656:	learn: 0.6021542	total: 30s	remaining: 15.6s
657:	learn: 0.6021375	total: 30s	remaining: 15.6s
658:	learn: 0.6021359	total: 30.1s	remaining: 15.6s
659:	learn: 0.6019521	total: 30.1s	remaining: 15.5s
660:	learn: 0.6019028	total: 30.1s	remaining: 15.5s
661:	learn: 0.6017303	total: 30.2s	remaining: 15.4s
662:	learn: 0.6017302	total: 30.2s	remaining: 15.4s
663:	learn: 0.6015335	total: 30.3s	remaining: 15.3s
664:	learn: 0.6015113	total: 30.3s	remaining: 15.3s
665:	learn: 0.601479

805:	learn: 0.5904531	total: 36.4s	remaining: 8.76s
806:	learn: 0.5903084	total: 36.4s	remaining: 8.72s
807:	learn: 0.5903065	total: 36.5s	remaining: 8.67s
808:	learn: 0.5901396	total: 36.5s	remaining: 8.63s
809:	learn: 0.5901395	total: 36.6s	remaining: 8.57s
810:	learn: 0.5901236	total: 36.6s	remaining: 8.53s
811:	learn: 0.5900080	total: 36.6s	remaining: 8.48s
812:	learn: 0.5900079	total: 36.7s	remaining: 8.44s
813:	learn: 0.5900069	total: 36.7s	remaining: 8.39s
814:	learn: 0.5899894	total: 36.7s	remaining: 8.34s
815:	learn: 0.5898501	total: 36.8s	remaining: 8.3s
816:	learn: 0.5898182	total: 36.8s	remaining: 8.25s
817:	learn: 0.5896201	total: 36.9s	remaining: 8.21s
818:	learn: 0.5896196	total: 36.9s	remaining: 8.16s
819:	learn: 0.5895266	total: 37s	remaining: 8.11s
820:	learn: 0.5895264	total: 37s	remaining: 8.06s
821:	learn: 0.5895246	total: 37s	remaining: 8.02s
822:	learn: 0.5894133	total: 37.1s	remaining: 7.97s
823:	learn: 0.5892771	total: 37.1s	remaining: 7.93s
824:	learn: 0.58927

964:	learn: 0.5799787	total: 42.9s	remaining: 1.55s
965:	learn: 0.5799781	total: 42.9s	remaining: 1.51s
966:	learn: 0.5799694	total: 43s	remaining: 1.47s
967:	learn: 0.5799665	total: 43s	remaining: 1.42s
968:	learn: 0.5797842	total: 43.1s	remaining: 1.38s
969:	learn: 0.5797108	total: 43.1s	remaining: 1.33s
970:	learn: 0.5796119	total: 43.2s	remaining: 1.29s
971:	learn: 0.5795249	total: 43.2s	remaining: 1.24s
972:	learn: 0.5795221	total: 43.3s	remaining: 1.2s
973:	learn: 0.5794799	total: 43.3s	remaining: 1.16s
974:	learn: 0.5793364	total: 43.4s	remaining: 1.11s
975:	learn: 0.5792166	total: 43.4s	remaining: 1.07s
976:	learn: 0.5792110	total: 43.5s	remaining: 1.02s
977:	learn: 0.5792107	total: 43.5s	remaining: 978ms
978:	learn: 0.5791384	total: 43.5s	remaining: 934ms
979:	learn: 0.5791368	total: 43.6s	remaining: 889ms
980:	learn: 0.5791342	total: 43.6s	remaining: 845ms
981:	learn: 0.5790271	total: 43.7s	remaining: 800ms
982:	learn: 0.5790263	total: 43.7s	remaining: 756ms
983:	learn: 0.579

## evaluation

In [15]:
y_test_pred = ctb_model.predict_proba(X_test)[:,1]
y_test_pred_label = ctb_model.predict(X_test)

fpr, tpr, thresholds = roc_curve(
    y_test,
    y_test_pred, 
    pos_label=1)
test_auc_ctb = auc(fpr, tpr)

test_average_precision_ctb = average_precision_score(
    y_test,
    y_test_pred, 
    )

test_acc_ctb = average_precision_score(
    y_test,
    y_test_pred_label, 
    )

test_f1_ctb = f1_score(
    y_test,
    y_test_pred_label, 
    )

test_recall_ctb = recall_score(
    y_test,
    y_test_pred_label, 
    )

test_precision_ctb = precision_score(
    y_test,
    y_test_pred_label, 
    )

In [16]:
y_training_pred = ctb_model.predict_proba(X_training)[:,1]
y_training_pred_label = ctb_model.predict(X_training)

fpr, tpr, thresholds = roc_curve(
    y_training,
    y_training_pred, 
    pos_label=1)
training_auc_ctb = auc(fpr, tpr)

training_average_precision_ctb = average_precision_score(
    y_training,
    y_training_pred, 
    )

training_acc_ctb = average_precision_score(
    y_training,
    y_training_pred_label, 
    )

training_f1_ctb = f1_score(
    y_training,
    y_training_pred_label, 
    )

training_recall_ctb = recall_score(
    y_training,
    y_training_pred_label, 
    )

training_precision_ctb = precision_score(
    y_training,
    y_training_pred_label, 
    )

In [17]:
metric_df = [
    {
    'metric':'AUC',
    'training':training_auc_ctb,
    'test':test_auc_ctb
    },
    {
    'metric':'Average Precision',
    'training':training_average_precision_ctb,
    'test':test_average_precision_ctb
    },  
    {
    'metric':'Accuracy',
    'training':training_acc_ctb,
    'test':test_acc_ctb
    }, 
    {
    'metric':'F1',
    'training':training_f1_ctb,
    'test':test_f1_ctb
    },
    {
    'metric':'Recall',
    'training':training_recall_ctb,
    'test':test_recall_ctb
    },
    {
    'metric':'Precision',
    'training':training_precision_ctb,
    'test':test_precision_ctb
    },
]

pd.DataFrame(metric_df)

,metric,training,test
0,AUC,0.880855,0.494023
1,Average Precision,0.897242,0.523570
2,Accuracy,0.702512,0.521604
3,F1,0.810462,0.661568
4,Recall,0.938800,0.901042
5,Precision,0.712993,0.522659


In [18]:
test_positive_rate = np.sum(y_test)/len(y_test)
print(f"test_positive_rate:{test_positive_rate:0.4f}")

test_positive_rate:0.5120


## feature importance

In [19]:
feature_importance = []

for n, i in zip(ctb_model.feature_names_, ctb_model.feature_importances_):
    feature_importance.append({
        'feature_name':n,
        'feature_importance':i
    })
    
pd.DataFrame(feature_importance).to_sql(
    "feature_importance_catboost_rise", 
    conn,
    if_exists = "replace",
    )

72

In [20]:
sql(f"""

with total_weight as (
select sum(feature_importance) as feature_importance
from feature_importance_catboost_rise
)

select a.*,
a.feature_importance/b.feature_importance as feature_importance_percentage
from feature_importance_catboost_rise as a,
total_weight as b
--where a.feature_name like '%rise%'
order by a.feature_importance desc
""")

,index,feature_name,feature_importance,feature_importance_percentage
0,13,feature_num__close_to_open_ratio__8_day_ago,6.169894,0.061699
1,61,feature_num__close_to_open_ratio__2_day_ago,5.693423,0.056934
2,69,feature_num__close_to_open_ratio__1_day_ago,5.335651,0.053357
3,46,feature_num__high_to_low_ratio__4_day_ago,4.716499,0.047165
4,21,feature_num__close_to_open_ratio__7_day_ago,3.754722,0.037547
...,...,...,...,...
67,19,feature_num__Low__7_day_ago,0.153178,0.001532
68,2,feature_num__High__9_day_ago,0.143747,0.001437
69,40,feature_num__Open__4_day_ago,0.136201,0.001362
70,16,feature_num__Open__7_day_ago,0.102918,0.001029


# end